In [28]:
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime

base_url = 'https://bachtrack.com/find-reviews/category=2;medium=1,3;startrow='
reviews=[]

In [37]:
#Move in increments of 50

for page in range(74,78):

    if page%10==0:
        print(page)
    
    url = base_url + str(page*50)
    r  = requests.get(url)
    soup = BeautifulSoup(r.text, "html5lib")

    listings = soup.find_all('div', attrs={"class": "article-shortform"})        

    for listing in listings:
        
        review = [] 

        event  = listing.find(itemprop="itemReviewed")
        event_url = event.find(itemprop="url").get_text()
        location  = event.find(itemprop="location").find(itemprop="name").get_text()    
        author    = listing.find(itemprop="author").find(itemprop="name").get_text()    
        title     = listing.find(itemprop="headline").get_text()    
        try:
            rating = listing.find(itemprop="ratingValue").get_text()
        except:
            rating = ''
        date      = listing.find(itemprop="datePublished").get_text()
        lang      = listing.find(itemprop="inLanguage").get_text()
        url       = listing.find(itemprop="mainEntityOfPage").get_text()
        
        #Get review's programme from listing
        r  = requests.get(event_url)
        soup = BeautifulSoup(r.text, "html5lib")  
        programme = soup.find(attrs={'id': 'tbody_listing-programme'})
        if programme == None:
            work = ''
            composer = ''            
        else:    
            for row in programme.find_all('tr'):
                detail = []
                for item in row.find_all('a'):
                    if '/composer/' in item.get('href'):
                        composer = item.get_text()
                        try:
                            composer = composer.replace(' ' + re.search('\((.*?)\)', composer).group(0), '')
                        except:
                            None
                        break
                for item in row.find_all('a'):
                    if '/work/' in item.get('href'):
                        work = item.get_text()
                        break
        
        #Get review text
        r  = requests.get(url)
        soup = BeautifulSoup(r.text, "html5lib")  
        if 'not found' not in soup.title.get_text().lower():
            
            review_bod = soup.find(itemprop="reviewBody").find_all('p')
            full_text=''
            for w in review_bod:
                if w.get('class')==None:
                    full_text += w.get_text()        

            review.append(composer)
            review.append(work)        
            review.append(title)
            review.append(location)
            review.append(author)
            review.append(event_url)
            review.append(rating)
            review.append(date)
            review.append(lang)
            review.append(full_text)
    #         print(review)        
            reviews.append(review)        

        
import pandas as pd                     
headers = ['composer', 'work', 'title', 'venue', 'author', 'url', 'rating', 'date', 'lang', 'full_text']
df = pd.DataFrame(reviews, columns=headers)
df = df.drop_duplicates()
df.to_csv('../../data/processed/reviews/bachtrack.csv')